# 🔎 Content Decay Finder & Refresh Planner

Automatically detect pages whose organic traffic from Google Search has **decayed**, quantify lost clicks, and generate a prioritized refresh plan — all powered by the Google Search Console API.

## Requirements
1. **Google Cloud project** with the *Search Console API* enabled.
2. **Service‑account JSON** added as a user to the Search Console property.
3. Python packages: `google-api-python-client`, `pandas`, `numpy`, `plotly`, `tqdm`.

> **Tip**: In Colab, you can upload the JSON key via the sidebar **Files → Upload** and set its path below.

In [ ]:
# Install deps (Colab only — comment out locally)
!pip -q install google-api-python-client tqdm plotly pandas numpy python-dateutil

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
from google.oauth2 import service_account
from googleapiclient.discovery import build
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.float_format = '{:,.2f}'.format

### 🔧 Configuration
Fill in your property URL and path to the service‑account JSON. Tweak the analysis window and decay threshold if needed.

In [ ]:
# --- USER CONFIG ---
PROPERTY_URL = 'https://www.example.com'  # Your exact Search Console property URL or 'sc-domain:example.com'
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/creds/sc_service_account.json'

LOOKBACK_DAYS = 540          # Pull ~18 months of data
MOVING_AVG_WEEKS = 12         # Trailing window for moving average
DECAY_THRESHOLD = 0.20        # 20% drop triggers flag

# --------------------
END_DATE = date.today()
START_DATE = END_DATE - timedelta(days=LOOKBACK_DAYS)


### 🔑 Authenticate & build Search Console service

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly']
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
service = build('searchconsole', 'v1', credentials=creds)
print('✅ Authenticated')

### ⬇️ Pull daily Search Console data

In [ ]:
def fetch_daily_data(property_url: str, start_date: date, end_date: date) -> pd.DataFrame:
    """Fetch daily clicks & impressions from GSC."""
    all_rows = []
    request = {
        'startDate': start_date.isoformat(),
        'endDate': end_date.isoformat(),
        'dimensions': ['page', 'date'],
        'rowLimit': 25000
    }
    response = service.searchanalytics().query(siteUrl=property_url, body=request).execute()
    rows = response.get('rows', [])
    for row in rows:
        all_rows.append({
            'url': row['keys'][0],
            'date': row['keys'][1],
            'clicks': row['clicks'],
            'impressions': row['impressions']
        })
    df = pd.DataFrame(all_rows)
    df['date'] = pd.to_datetime(df['date'])
    return df

df_daily = fetch_daily_data(PROPERTY_URL, START_DATE, END_DATE)
print(f'Pulled {len(df_daily):,} daily rows for {df_daily.url.nunique():,} pages')
df_daily.head()

### 📊 Aggregate to weekly & compute decay

In [ ]:
# Resample to weekly clicks
df_weekly = (
    df_daily
    .set_index('date')
    .groupby('url')
    .resample('W')
    .sum()
    .reset_index()
)

# Compute trailing moving average & previous-period average
def detect_decay(df_group):
    df_group = df_group.sort_values('date')
    df_group['ma'] = df_group['clicks'].rolling(window=MOVING_AVG_WEEKS, min_periods=MOVING_AVG_WEEKS).mean()
    # Compare last MA vs same length preceding period
    if len(df_group) < MOVING_AVG_WEEKS * 2:
        return None  # not enough data
    recent_avg = df_group['ma'].iloc[-1]
    prior_avg = df_group['ma'].iloc[-MOVING_AVG_WEEKS-1]
    decay_pct = (prior_avg - recent_avg) / prior_avg if prior_avg > 0 else 0
    lost_clicks = (prior_avg - recent_avg) * MOVING_AVG_WEEKS
    return pd.Series({'recent_avg': recent_avg, 'prior_avg': prior_avg, 'decay_pct': decay_pct, 'lost_clicks': lost_clicks})

decay_records = []
for url, grp in df_weekly.groupby('url'):
    rec = detect_decay(grp)
    if rec is not None:
        rec['url'] = url
        decay_records.append(rec)

df_decay = pd.DataFrame(decay_records)
df_decay = df_decay[df_decay['decay_pct'] >= DECAY_THRESHOLD]
df_decay = df_decay.sort_values('lost_clicks', ascending=False).reset_index(drop=True)
print(f'Pages flagged: {len(df_decay):,}')
df_decay.head()

### 📈 Visualise lost clicks by URL

In [ ]:
fig = px.bar(
    df_decay.head(50),
    x='lost_clicks',
    y='url',
    orientation='h',
    title='Top pages by lost clicks',
    labels={'lost_clicks': 'Lost clicks (last period vs. previous)', 'url': ''}
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

### 💾 Export results

In [ ]:
OUTPUT_CSV = '/content/decay_refresh_plan.csv'
df_decay.to_csv(OUTPUT_CSV, index=False)
print(f'✅ Saved to {OUTPUT_CSV}')